# <i>COM-418 - Computers and Music</i> <br /> <br />  <b>Physical modelling</b> 

<div align="right"><a href="https://people.epfl.ch/lucie.perrotta">Lucie Perrotta</a> and <a href="https://people.epfl.ch/paolo.prandoni">Paolo Prandoni</a>, <a href="https://www.epfl.ch/labs/lcav/">LCAV, EPFL</a></div>

In [113]:
%matplotlib inline
import ipywidgets as widgets

import numpy as np
from numpy.fft import fft, ifft
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy import signal
import import_ipynb
from Helpers import * 
import matplotlib
from IPython.display import IFrame

figsize=(10,5)
matplotlib.rcParams.update({'font.size': 16})

fs=44100;

## 1. Karplus–Strong string synthesis

A rather easy model for synthesizing a string was proposed in 1983 in the article "Digital Synthesis of Plucked String and Drum Timbres" by Karplus and Strong. It defines the string as being a very short noise generator followed by a lowpass loop. The delay of the loop determines the frequency of the string, and the length of the noise burst the "precision" of the picking of the string. Hence, a short noise burst would generate a rather "striked string" sound, corresponding to instruments such as the harpsicord, while longer noise burst would create a more "bowed string" sound, such as a violin.

The diagram is as follows

<img src="pictures/strong.png" alt="Drawing" style="width: 60%;"/>

where the lowpass filter is typically implement using a simple moving average over the past samples. The size of the moving average influences the dampering of the string over time.

The value $N$ (in samples) of the delay is simply calculated by (euclidean) dividing the sampling frequency $f_s$ by the desired string frequency $f$,
$$
N = \lfloor fs /f \rfloor .
$$

Let's implement it!

In [166]:
def karplus_strong(freq, pinch=.02, sustain=.99, damper=2, duration=2, lowpass=False):
    """
    Plays a string sound using the Karplus-Strong model.
    freq: the frequency (in hertz) of the played string
    pinch: the length (in seconds) of the initial pinching of the string
    sustain: the sustain factor of the vibration of the string after the pinching (1 is normal sustain)
    damper: the level of dampering of the string (1 = no damper, higher int values is more damper)
    duration: the duration of the output sound in seconds
    lowpass: apply a lowpass to the output signal
    returns: a numpy array containing the vibrating string's data
    """
    out = np.zeros(duration*fs) # placeholder for the output
    burst = 2*np.random.random(int(pinch*fs)) - 1 # Random noise in [-1, 1] lasting pinch ms
    out[:int(pinch*fs)] = burst # output starts with burst
    
    delay = fs//freq # delay of the loop (in samples)
    for i in range(delay, duration*fs): # delay loop
        out[i] += sustain * np.mean(out[i-delay:i-delay+damper])
        
    if lowpass: out = butter_pass_filter(out, np.array([int(8*freq)]), 10000, "low", order=3)
    return normalize(out)

Let's test it with a 200Hz frequency.

In [170]:
string = karplus_strong(150)

Audio(string, rate=fs)

Sounds quite synthetic, but given the simplycity of the code above, this is already quite good! Let's have a look at the frequencies and try playing with them.

In [157]:
@widgets.interact(freq=(50, 500, 50), pinch=(0.1, 0.5, 0.1), sustain=(.98, 1, 0.001), damper=(1, 10, 1))
def update(freq=200, pinch=.02, sustain=.99, damper=2,):
    
    test = karplus_strong(freq, pinch, sustain, damper, duration=2)

    plt.figure(figsize=figsize)
    plt.plot(test)
    plt.xlabel("Time [samples]")
    plt.show()

    plt.figure(figsize=figsize)
    plt.magnitude_spectrum(test)
    plt.xlim(0, .3)
    plt.show()

interactive(children=(IntSlider(value=200, description='freq', max=500, min=50, step=50), FloatSlider(value=0.…

The frequency spectrum is quite rich already. Try playing with the sliders and observe how each parameter influences the spectrum.